In [53]:
# Import Statements
import re
import os
import csv
import gzip
import copy
import json
import spacy
import string
import requests
import numpy as np
import pandas as pd
import en_core_web_sm
from afinn import Afinn
from pathlib import Path
from spacy import displacy
from normalise import normalise
from normalise import tokenize_basic
from autocorrect import Speller
from collections import Counter
from gensim import corpora, models
import nltk
from nltk.tree import Tree
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from nltk.tag import StanfordNERTagger
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.chunk import conlltags2tree, tree2conlltags
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer

### LOAD CONFIGURATION LIBARIES
java_path = "C:/Program Files/Java/jdk1.8.0_231/bin/java.exe"
os.environ['JAVAHOME'] = java_path
nlp = en_core_web_sm.load()
st = StanfordNERTagger('C:/Users/NA29187/Downloads/stanford-ner-2018-10-16/stanford-ner-2018-10-16/classifiers/english.all.3class.distsim.crf.ser.gz',
					   'C:/Users/NA29187/Downloads/stanford-ner-2018-10-16/stanford-ner-2018-10-16/stanford-ner.jar',
					   encoding='utf-8')
"""
nltk.download('punkt') -> missing tokenzer, might need installing
for dependency in ("words", brown", "names", "wordnet", "averaged_perceptron_tagger", "universal_tagset", "stop_words", "maxent_ne_chunker", "punkt"):
    nltk.download(dependency)
"""

'\nnltk.download(\'punkt\') -> missing tokenzer, might need installing\nfor dependency in ("words", brown", "names", "wordnet", "averaged_perceptron_tagger", "universal_tagset", "stop_words", "maxent_ne_chunker", "punkt"):\n    nltk.download(dependency)\n'

In [26]:
slangDict = {
    'afaik': 'as far as i know',
    'bc': 'because',
    'bfn': 'bye for now',
    'br': 'best regards',
    'btw': 'by the way',
    'dm': 'direct message',
    'em': 'email',
    'fb': 'facebook',
    'ffs': 'for fucks sake',
    'fml': 'fuck my life',
    'rt': 'retweet',
    'bgd': 'background',
    'ab': 'about',
    'abt': 'about',
    'dd': 'dear daughter',
    'ayfkmwts': 'are you fucking kidding me with this shit',
    'w': 'with',
    'wo': 'without',
    'dyk': 'did you know',
    'eli5': 'explain it to me like i am five',
    'fbf': 'flashback friday',
    'fomo': 'fear of missing out',
    'ftw': 'for the win',
    'fyi': 'for your information',
    'icymi': 'in case you missed it',
    'ht': 'hat tip',
    'imo': 'in my opinion',
    'imho': 'in my humble opinion',
    'irl': 'in real life',
    'lmk': 'let me know',
    'nbd': 'no big deal',
    'nsfw': 'not safe for work',
    'wfh': 'working from home',
    'covid': 'coronavirus',
    'smh': 'shaking my head',
    'tbh': 'to be honest',
    'tbt': 'throwback thursday',
    'tfw': 'that feeling when',
    'tgif': 'thank god it is friday',
    'tldr': 'too long did not read',
    'wbw': 'way back wednesday',
    'af': 'as fuck',
    'bae': 'my crush',
    'hmu': 'hit me up',
    'idk': 'i do not know',
    'idc': 'i do not care',
    'fwiw': 'for what it is worth',
    'ily': 'i love you',
    'iso': 'in search of',
    'jk': 'just kidding',
    'jtm': 'just the messenger',
    'lmao': 'laughing my ass off',
    'lol': 'laughing out loud',
    'rip': 'rest in peace',
    'nvm': 'nevermind',
    'obv': 'obviously',
    'omg': 'oh my god',
    'omw': 'on my way',
    'pls': 'please',
    'psa': 'public service announcement',
    'rn': 'right now',
    'rofl': 'rolling on the floor laughing',
    'srsly': 'seriously',
    'til': 'today i learned',
    'tmi': 'too much information',
    'ty': 'thank you',
    'wtf': 'what the fuck',
    'yw': 'you are welcome',
    'bfn': 'bye for now',
    'da': 'the',
    'deets': 'details',
    'fab': 'fabulous',
    'fav': 'favorite',
    'ic': 'i see',
    'kk': 'cool cool',
    'nts': 'note to self',
    'prt': 'please retweet',
    'tftf': 'thank for the follow',
    'tmb': 'tweet me back',
    'u': 'you',
    'woz': 'was',
    'wtv': 'whatever',
    'ykyat': 'you know you are addicted to',
    'yolo': 'you only live once',
    'yoyo': 'you are on your own'
}

In [27]:
def generate_corpus(maxNum=1000):
    ''' Utilize the get old tweets package to aggregate historical tweets - easier to use than Twitter API '''
    # Build the OS call to generate the csv
    FILE_PATH = 'C:/Users/NA29187/Downloads/Optimized-Modified-GetOldTweets3-OMGOT-master/Optimized-Modified-GetOldTweets3-OMGOT-master/GetOldTweets3-0.0.10/'
    SCRIPT = 'GetOldTweets3.py '
    PYTHON_PATH = 'C:/ProgramData/Anaconda3/python.exe '
    QUERY = '--near "42.376, -71.1097" --within 40km --maxtweets {}'.format(maxNum)
    print('Executing Query: {}'.format(PYTHON_PATH + FILE_PATH + SCRIPT + QUERY))
    os.system(PYTHON_PATH + FILE_PATH + SCRIPT + QUERY)

In [28]:
def read_corpus():
    ''' Read in downloaded tweet meta data and extract key fields '''
    data = pd.read_csv("output_got.csv")
    # Remove useless columns to save space in memory
    del data['replies']
    del data['retweets']
    del data['favorites']
    del data['hashtags']
    del data['permalink']
    # Create area to store cleaned tweet
    data['cleaned_tweet'] = data['text']
    data['tokens'] = ''
    data['parts_of_speech'] = ''
    data['chunks'] = ''
    data['entities'] = ''
    return data

In [7]:
def clean_text(tweet):
    ''' Basic steps to clean text and strip out conflicting elements '''
    try:
        # Convert text to lowercase
        tweet = tweet.lower()
        # Remove hyperlinks
        tweet = re.sub(r'^https?:\/\/.*[\r\n]*', '', tweet)
        tweet = re.sub(r'^http?:\/\/.*[\r\n]*', '', tweet)
        # Remove puntucation
        tweet = tweet.translate(str.maketrans('', '', string.punctuation))
        # Remove white space
        tweet = tweet.strip()
        # Remove non ascii characters
        tweet = ascii_normalize(tweet)
        # Return Finished Product
        return tweet
    except Exception as e:
        return ''

def spell_check(tweet_words):
    ''' Auto correct tweets according to english language standards '''
    # Tweets are incredibly irregular and often quickly typed
    spell = Speller(lang='en')
    return [spell(word) for word in tweet_words]

def remove_slang(tweet_words):
    ''' Remove the slang words from dictionary '''
    outData = []
    for word in tweet_words:
        outData.append(slangDict.get(word, word))
    return outData

def ascii_normalize(tweet):
    ''' Remove non ascii characters '''
    encodeString = tweet.encode("ascii", "ignore")
    return encodeString.decode()
    
def tokenize_and_stop(tweet):
    ''' Convert string to tokens and remove stop words '''
    tweet_tokenizer = TweetTokenizer()
    tokens = tweet_tokenizer.tokenize(tweet)
    stopWords = set(stopwords.words('english'))
    cleaned = [t for t in tokens if t not in stopWords]
    return cleaned

def lem_tokens(tokens):
    ''' Lemmanization, extract root words '''
    lemmatizer=WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in tokens]

def pos_tag(tokens):
    ''' Associate part of speech with the word tokens '''
    return nltk.pos_tag(tokens)
    
def chunking(posRecords):
    ''' Apply chunking on in appropriate segements (prestep for NLP) '''
    pattern = 'NP: {<DT>?<JJ>*<NN>}'
    chunkParser = nltk.RegexpParser(pattern)
    chunks = chunkParser.parse(posRecords)
    # IOB = Inside, outside, beginning - great for understanding where chunk comes
    iob_tagged = tree2conlltags(chunks)
    return iob_tagged

def named_entity_recognition_master(pos, tokens, cleanedTweet):
    ''' Perform named entity recognition on the part of speech tagged chunks '''
    usefulTags= ['WORK_OF_ART','DATE','LOC','PERSON','LOCATION','ORGANIZATION','GEO', 'ORG','PER', 'GPE','TIME','EVB']
    # NTLK
    nltkEntities = nltk.ne_chunk(pos)
    # Spacey
    spaceyEntities = spacey_tokenize(cleanedTweet)
    # Stanford NLP
    stanfordEntities = []#stanford_tokenize(tokens)
    outData = set()
    # Cross check that we only pull out relevant tags
    for val in spaceyEntities:
        if val[1] in usefulTags:
            outData.add(val)
    for val in stanfordEntities:
        if val[1] in usefulTags:
            outData.add(val)
    for val in get_continuous_chunks(nltkEntities):
        if val[1] in usefulTags:
            outData.add(val)
    return copy.deepcopy(outData)

def nltK_tokenize(posTags):
    ''' Return nltk tokens from the part of speech tags '''
    return nltk.ne_chunk(posTags)
    
def spacey_tokenize(tweet):
    ''' Return spacey named entities from the cleaned tweet '''
    doc = nlp(tweet)
    return [(X.text, X.label_) for X in doc.ents]

def stanford_tokenize(tokens):
    ''' Use the stanford classification libarary to parse out locations '''
    return st.tag(tokenized_text)
    

def token_pipeline(tweet):
    ''' Pipeline to turn cleaned tweet into tokens '''
    tokens = tokenize_and_stop(tweet)
    tokens = remove_slang(tokens)
    tokens = spell_check(tokens)
    tokens = lem_tokens(tokens)
    return tokens

def pipeline(record):
    ''' Main pipeline to ingest, clean, and process the tweets'''
    tweet = record.get('tweet')
    tweet = cleanText(tweet)
    tokens = tokenizeAndStop(tweet)
    tokens = spell_check(tokens)
    tokens = lemTokens(tokens)
    pos = posTag(tokens)
    chunks = chunking(pos)
    names = namedEntityRecognition(pos)
    record['parsedTweet'] = tweet
    record['tokens'] = tokens
    record['pos'] = pos
    record['chunks'] = chunks
    record['ner'] = names
    return record
    
def get_continuous_chunks(chunked):
    ''' Extract entities from NLTK tree '''
    continuous_chunk = []
    current_chunk = []
    for i in chunked:
        if type(i) == Tree:
            print('trre!')
            current_chunk.append(" ".join([token for token, pos in i.leaves()]))
        elif current_chunk:
            named_entity = " ".join(current_chunk)
            if named_entity not in continuous_chunk:
                continuous_chunk.append(named_entity)
                current_chunk = []
            else:
                continue
    return continuous_chunk

In [9]:
#######################################
#         SEMANTIC ANALYSIS           #
#######################################
def semantic_analysis_initialization(num_topics):
    ''' TF-IDF analyis to extract key terms from each tweet with further enrichment of LDA '''
    dictionary_LDA = corpora.Dictionary(data['tokens'])
    dictionary_LDA.filter_extremes(no_below=3)
    corpus = [dictionary_LDA.doc2bow(list_of_tokens) for list_of_tokens in data['tokens']]

    %time lda_model = models.LdaModel(corpus, num_topics=num_topics, \
                                      id2word=dictionary_LDA, \
                                      passes=4, alpha=[0.01]*num_topics, \
                                      eta=[0.01]*len(dictionary_LDA.keys()))
    return lda_model

def apply_topics(tokens):
    global model
    potentialTopics = lda_model[dictionary_LDA.doc2bow(tokens)]
    # Sort topics
    potentialTopics = sorted(potentialTopics, key = lambda x: x[1])
    # Extract top 3 topics
    return [x[0] for x in potentialTopics[0:2]]

#######################################
#         SENTIMENT ANALYSIS          #
#######################################
def sentiment_analysis(tweet):
    ''' Simple Sentiment Analysis - Scored from -5 to 5 (negative to positive) '''
    afinn = Afinn()
    return afinn.score(tweet)

In [56]:
#######################################
#         GEOCODING SERVICES          #
#######################################

def googleEnrichments(entity):
    ''' Extract named entities and perform a location using google geolocation servies '''
    api_key = ''
    base_url = 'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?'
    # Now we need to define the parameters
    returnData = 'fields=formatted_address,name,geometry,types'
    url = base_url + 'key=' + api_key + '&' + 'inputtype=textquery&' + 'input=' + entity + '&' + returnData
    response = requests.get(url)
    return cleanGoogleResponse(response)

def cleanGoogleResponse(response):
    ''' Ingest response from Google API and extract the useable fields from it '''
    if response.status_code == 200:
        # Successful Request
        loadedData = json.loads(response.text)
        # Create Parsed location Data
        candidates = loadedData.get('candidates', [])
        # Remove viewport from the candidates array
        for x in range(len(candidates)):
            latLon = candidates[x]['geometry']['location']
            del candidates[x]['geometry']
            candidates[x]['location'] = latLon
        # Return the candidates array
        return candidates
    else:
        # Bad Request
        return []

In [3]:
#######################################
#         Deep Neural Network         #
#######################################
from keras.layers import  Dropout, Dense
from keras.models import Sequential
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn import metrics

def read_corpus_nn():
    ''' Read in downloaded tweet meta data and extract key fields '''
    data = pd.read_csv("labeled_data.csv")
    return data

def TFIDF(X_train, X_test,MAX_NB_WORDS=75000):
    vectorizer_x = TfidfVectorizer(max_features=MAX_NB_WORDS)
    X_train = vectorizer_x.fit_transform(X_train).toarray()
    X_test = vectorizer_x.transform(X_test).toarray()
    print("tf-idf with",str(np.array(X_train).shape[1]),"features")
    return (X_train,X_test)

def Build_Model_DNN_Text(shape, nClasses, dropout=0.5):
    """
    buildModel_DNN_Tex(shape, nClasses,dropout)
    Build Deep neural networks Model for text classification
    Shape is input feature space
    nClasses is number of classes
    """
    model = Sequential()
    node = 512 # number of nodes
    nLayers = 4 # number of  hidden layer
    model.add(Dense(node,input_dim=shape,activation='relu'))
    model.add(Dropout(dropout))
    for i in range(0,nLayers):
        model.add(Dense(node,input_dim=node,activation='relu'))
        model.add(Dropout(dropout))
    model.add(Dense(nClasses, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

Using TensorFlow backend.


In [ ]:
generate_corpus(500)
data= read_corpus()
data = data.iloc[:100]
data['cleaned_tweet']= data['cleaned_tweet'].apply(clean_text)
data['tokens'] = data['cleaned_tweet'].apply(token_pipeline)
data['parts_of_speech'] = data['tokens'].apply(pos_tag)
model = semantic_analysis_initialization(20)
topics = []
for i,topic in lda_model.show_topics(formatted=True, num_topics=num_topics, num_words=10):
    topics.append('Topics-' + str(i) + ': ' + topic)
# Initialize empty topics for each tweet
data['topics'] = data['tokens'].apply(apply_topics)
data['sentiment'] = data['cleaned_tweet'].apply(sentiment_analysis)
data['entities'] = data.apply(lambda row: named_entity_recognition_master(row['parts_of_speech'], row['tokens'], row['cleaned_tweet']), axis=1)
data.to_csv('out_data.csv')

In [76]:
### Deep Learning Execution ###
data_nn = read_corpus_nn()
# Extract only relevant information
data_nn = data_nn[['text', 'health rating']]
# Subset Data
data_nn = data_nn.iloc[:700]
# Fill any missing values
data_nn = data_nn.fillna(0)
# Run pipeline to generate additional input features
data_nn['cleaned_tweet'] = data_nn['text'].apply(clean_text)
# Run sentiment analysis of data
data_nn['sentiment'] = data_nn['cleaned_tweet'].apply(sentiment_analysis)
# Model split
X_train = data_nn['cleaned_tweet'].iloc[:600].astype(str)
X_test =  data_nn['cleaned_tweet'].iloc[600:].astype(str)
X_test.append(pd.Series("Went for a run along the Charles River today. OOF. Starting to feel sick, and starting to cough :)"))
Y_train = data_nn['health rating'].iloc[:600].astype(int)
Y_test =  data_nn['health rating'].iloc[600:].astype(int)
X_train_tfidf,X_test_tfidf = TFIDF(X_train,X_test)
# Zip tfidf with the sentiment again
X_train = list(zip(X_train_tfidf, data_nn['health rating'].iloc[600:]))
X_test = list(zip(X_test_tfidf, data_nn['health rating'].iloc[600:]))
model_DNN = Build_Model_DNN_Text(X_train_tfidf.shape[1], 5)
model_DNN.fit(X_train_tfidf, Y_train,
                              validation_data=(X_test_tfidf, Y_test),
                              epochs=25,
                              batch_size=128,
                              verbose=2)
predicted = model_DNN.predict_classes(X_test_tfidf)
print(metrics.classification_report(Y_test, predicted))


tf-idf with 3070 features
Train on 600 samples, validate on 100 samples
Epoch 1/25
 - 0s - loss: 1.5004 - accuracy: 0.5217 - val_loss: 1.3147 - val_accuracy: 0.5900
Epoch 2/25
 - 0s - loss: 1.2127 - accuracy: 0.6067 - val_loss: 1.2593 - val_accuracy: 0.5900
Epoch 3/25
 - 0s - loss: 1.1138 - accuracy: 0.6067 - val_loss: 1.2799 - val_accuracy: 0.5900
Epoch 4/25
 - 0s - loss: 1.0606 - accuracy: 0.6067 - val_loss: 1.2374 - val_accuracy: 0.5900
Epoch 5/25
 - 0s - loss: 0.9731 - accuracy: 0.6067 - val_loss: 1.1925 - val_accuracy: 0.5900
Epoch 6/25
 - 0s - loss: 0.8678 - accuracy: 0.6067 - val_loss: 1.1703 - val_accuracy: 0.5900
Epoch 7/25
 - 0s - loss: 0.7737 - accuracy: 0.6067 - val_loss: 1.2042 - val_accuracy: 0.5900
Epoch 8/25
 - 0s - loss: 0.6938 - accuracy: 0.6083 - val_loss: 1.1931 - val_accuracy: 0.5700
Epoch 9/25
 - 0s - loss: 0.6644 - accuracy: 0.6800 - val_loss: 1.2409 - val_accuracy: 0.5600
Epoch 10/25
 - 0s - loss: 0.6483 - accuracy: 0.7483 - val_loss: 1.3306 - val_accuracy: 0.56

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
